# Creating cluster member lists

Given a cluster ID, save its member list as a txt file.

In [47]:
import numpy as np

# path to cluster catalog
cluster_cat_path = "camira_s19a_wide_sm_v1.txt"
# path to cluster members catalog
cluster_mem_path = "camira_mem_s19a_wide_sm_v1.dat"

In [60]:
HSC_cl_cat = np.loadtxt(cluster_cat_path, usecols=(0,1,2,3,4,5)) 
HSC_cl_cat_IDs = np.loadtxt(cluster_cat_path, dtype='str', usecols=(6)) 

cluster_mems = np.loadtxt(cluster_mem_path)

## Dictionary of cluster coords to cluster ID

In [73]:
list_props_ID = []

for i in range(len(HSC_cl_cat)):
    RA = HSC_cl_cat[i, 0]
    dec = HSC_cl_cat[i, 1]
    # z in the full member list is rounded to 3 decimal places
    z = np.round(HSC_cl_cat[i, 2], 3)
    ID = HSC_cl_cat_IDs[i]
    props = (RA, dec, z)
    
    list_props_ID.append((props, ID))
    
dict_props_ID = dict(list_props_ID)

In [111]:
# this dictionary stores the key-value pair of (cluster ID)-slice(a, b), where: 
# 1. cluster ID is found via dict_props_ID
# 2. a is the first place in the full member list where the cluster RA appears 
# and b-1 is the last
list_ID_slice = []
low_ind = 0 # a
high_ind = 0 # b

num_of_cls = 0

for i in range(len(cluster_mems)):
    cluster_RA = cluster_mems[i, 0]
    cluster_dec = cluster_mems[i, 1]
    cluster_z = cluster_mems[i, 3]
    props = (cluster_RA, cluster_dec, cluster_z)
    
    if (i < (len(cluster_mems) - 1)): 
        if (cluster_RA == cluster_mems[i+1, 0]) and (cluster_z == cluster_mems[i+1, 3]): 
            continue

    # if we've reached this point, it means we've reached the last galaxy 
    # in this cluster
    try:
        ID = dict_props_ID[props]
    except: # account for rounding variation in z
        try:
            props = (cluster_RA, cluster_dec, np.round(cluster_z+0.001,3))
            ID = dict_props_ID[props]
        except:
            props = (cluster_RA, cluster_dec, np.round(cluster_z-0.001,3))
            ID = dict_props_ID[props]
    
    high_ind = i + 1
    list_ID_slice.append((ID, slice(low_ind, high_ind)))
    low_ind = high_ind
    
dict_ID_slice = dict(list_ID_slice)

In [112]:
print(f'number of clusters: {len(dict_props_ID)}')
print(f'size of dict: {len(dict_ID_slice)}')

number of clusters: 16589
size of dict: 16589


In [123]:
ID = 'C19a1sW02869'
prob_cutoff = 0.
path = 'Member_Lists/memlist_' + ID + '_aboveP=_' + str(prob_cutoff)
member_slice = dict_ID_slice[ID]

mem_list_no_cut = cluster_mems[member_slice]
above_prob_cutoff = members[:,7] > prob_cutoff
mem_list = mem_list_no_cut[above_prob_cutoff]

np.savetxt(fname=path, X=mem_list)